In [82]:
import torch
import os
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from more_itertools import chunked
from tqdm import tqdm
import numpy as np
from scipy.stats import t

In [83]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [84]:
device

device(type='cuda')

In [85]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [86]:
model_name_or_path = 'tinkoff-ai/response-quality-classifier-large'
file_with_dialogs = 'testR+sftw.csv'
CONFIDENCE_LEVEL = 0.99

In [87]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path)
model = model_met.to(device)

In [88]:
df = pd.read_csv(file_with_dialogs, sep=',')

In [89]:
df

,Unnamed: 0,Instruction,sft_model,Ans RetrivalR,Ans ClassicR
0,0,Как улучшить свои навыки управления временем (...,Как улучшить свои навыки управления временем (...,Как улучшить свои навыки управления временем (...,Как улучшить свои навыки управления временем (...
1,1,Какие самые эффективные способы борьбы со стре...,Какие самые эффективные способы борьбы со стре...,Какие самые эффективные способы борьбы со стре...,Какие самые эффективные способы борьбы со стре...
2,2,В чем основные различия между языками программ...,В чем основные различия между языками программ...,В чем основные различия между языками программ...,В чем основные различия между языками программ...
3,3,Как я могу повысить свою эффективность на удал...,Как я могу повысить свою эффективность на удал...,Как я могу повысить свою эффективность на удал...,Как я могу повысить свою эффективность на удал...
4,4,Можешь объяснить основы квантовых вычислений?,Можешь объяснить основы квантовых вычислений? ...,Можешь объяснить основы квантовых вычислений? ...,Можешь объяснить основы квантовых вычислений? ...
...,...,...,...,...,...
88,88,Что такое Kmeans?,Что такое Kmeans? <instructionS> KMEAN - это а...,Что такое Kmeans? <instructionS> KME - это аб...,Что такое Kmeans? <instructionS> KM - это абб...
89,89,Чем отличается kmeans и knn?,Чем отличается kmeans и knn? <instructionS> KM...,Чем отличается kmeans и knn? <instructionS> К...,Чем отличается kmeans и knn? <instructionS> К...
90,90,Что такое угол эйлера?,Что такое угол эйлера? <instructionS> Углы Эйл...,Что такое угол эйлера? <instructionS> Угловой...,Что такое угол эйлера? <instructionS> Угловой...
91,91,Напиши о пользе водки,Напиши о пользе водки для здоровья. <instructi...,Напиши о пользе водки. <instructionS> Водка -...,Напиши о пользе водки. <instructionS> Водка -...


In [90]:
def fix_dialogs(df):
    dialogs_sft = []
    dialogs_retrival = []
    dialogs_classic = []
    for col, data in df.iterrows():
        dialogs_sft += [f'[CLS]{data["Instruction"]}[RESPONSE_TOKEN]{data["sft_model"]}']
        dialogs_retrival += [f'[CLS]{data["Instruction"]}[RESPONSE_TOKEN]{data["Ans RetrivalR"]}']
        dialogs_classic += [f'[CLS]{data["Instruction"]}[RESPONSE_TOKEN]{data["Ans ClassicR"]}']
    return dialogs_sft, dialogs_retrival, dialogs_classic

In [91]:
dialogs_sft, dialogs_retrival, dialogs_classic = fix_dialogs(df)

In [92]:
relevance_sft = []
specificity_sft = []
for batch in tqdm(dialogs_sft):
    inputs = tokenizer(batch, max_length=128, add_special_tokens=False, return_tensors='pt').to(device)
    with torch.inference_mode():
        logits = model(**inputs).logits
        probas = torch.sigmoid(logits)[0]
    relevance, specificity = probas.detach().cpu().numpy()
    relevance_sft = np.append(relevance_sft, relevance)
    specificity_sft = np.append(specificity_sft, specificity)

100%|██████████| 93/93 [00:01<00:00, 47.96it/s]


In [95]:
relevance_retrival = []
specificity_retrival = []
for batch in tqdm(dialogs_retrival):
    inputs = tokenizer(batch, max_length=128, add_special_tokens=False, return_tensors='pt').to(device)
    with torch.inference_mode():
        logits = model(**inputs).logits
        probas = torch.sigmoid(logits)[0]
    relevance, specificity = probas.detach().cpu().numpy()
    relevance_retrival = np.append(relevance_retrival, relevance)
    specificity_retrival = np.append(specificity_retrival, specificity)

100%|██████████| 93/93 [00:01<00:00, 48.70it/s]


In [96]:
relevance_classic = []
specificity_classic = []
for batch in tqdm(dialogs_classic):
    inputs = tokenizer(batch, max_length=128, add_special_tokens=False, return_tensors='pt').to(device)
    with torch.inference_mode():
        logits = model(**inputs).logits
        probas = torch.sigmoid(logits)[0]
    relevance, specificity = probas.detach().cpu().numpy()
    relevance_classic = np.append(relevance_classic, relevance)
    specificity_classic = np.append(specificity_classic, specificity)

100%|██████████| 93/93 [00:01<00:00, 48.85it/s]


In [63]:
def confidence_interval(data, confidence_level=CONFIDENCE_LEVEL):
    n = len(data)
    df = n - 1
    sem = np.std(data, ddof=1) / np.sqrt(n)

    margin_of_error = t.ppf((1 + confidence_level) / 2, df) * sem

    return margin_of_error

In [98]:
datadict = {}

In [97]:
def calc_mean_and_var(autoSSA_list, datadict, key):
    mean_ssa = round(np.mean(autoSSA_list), 3)
    var_ssa = confidence_interval(autoSSA_list)
    datadict[key] = f"{mean_ssa} ± {round(var_ssa, 3)}"
    return datadict

In [99]:
datadict = calc_mean_and_var(relevance_sft, datadict, "relevance_sft")

In [100]:
datadict = calc_mean_and_var(specificity_sft, datadict, "specificity_sft")

In [101]:
datadict = calc_mean_and_var(relevance_retrival, datadict, "relevance_retrival")

In [102]:
datadict = calc_mean_and_var(specificity_retrival, datadict, "specificity_retrival")

In [103]:
datadict = calc_mean_and_var(relevance_classic, datadict, "relevance_classic")

In [104]:
datadict = calc_mean_and_var(specificity_classic, datadict, "specificity_classic")

In [105]:
datadict

{'relevance_sft': '0.805 ± 0.056',
 'specificity_sft': '0.818 ± 0.05',
 'relevance_retrival': '0.82 ± 0.057',
 'specificity_retrival': '0.836 ± 0.047',
 'relevance_classic': '0.793 ± 0.071',
 'specificity_classic': '0.804 ± 0.064'}